# TDE 3 - Big Data Processing with Apache Spark

## Imports & Configs

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark import SparkFiles
from pyspark.sql import SparkSession

# SparkSQL
import pyspark.sql.functions as f

## Coletando os dados

In [ ]:
!wget -nc 'http://www.ppgia.pucpr.br/~jean.barddal/bigdata/transactions.csv.zip'
!unzip -n transactions.csv.zip

File ‘transactions.csv.zip’ already there; not retrieving.

Archive:  transactions.csv.zip


## Iniciando a sessão `spark`

In [ ]:
spark = SparkSession.builder.appName('Transactions').getOrCreate()
sc = spark.sparkContext

## Formato `RDD`

In [ ]:
rdd = sc.textFile("transactions.csv") \
        .map(lambda x: x.split(";")) \
        .filter(lambda a: a[0] != "country_or_area") \
        .filter(lambda a: a[2] != 'TOTAL')

In [1]:
# Visualizando as 5 primeiras instâncias
#rdd.take(5)

## Formato `csv` para SparkSQL

In [ ]:
df = spark.read.csv("transactions.csv", header=True, inferSchema=True, sep=";")

In [ ]:
# Configuração para realizar consultas com SQL
df.createOrReplaceTempView('table')

In [ ]:
# Visualizando as 5 primeiras instâncias
df.show(5)

+---------------+----+---------+--------------------+------+---------+---------+---------------+--------+---------------+
|country_or_area|year|comm_code|           commodity|  flow|trade_usd|weight_kg|  quantity_name|quantity|       category|
+---------------+----+---------+--------------------+------+---------+---------+---------------+--------+---------------+
|    Afghanistan|2016|   010410|         Sheep, live|Export|     6088|     2339|Number of items|    51.0|01_live_animals|
|    Afghanistan|2016|   010420|         Goats, live|Export|     3958|      984|Number of items|    53.0|01_live_animals|
|    Afghanistan|2008|   010210|Bovine animals, l...|Import|  1026804|      272|Number of items|  3769.0|01_live_animals|
|        Albania|2016|   010290|Bovine animals, l...|Import|  2414533|  1114023|Number of items|  6853.0|01_live_animals|
|        Albania|2016|   010392|Swine, live excep...|Import| 14265937|  9484953|Number of items| 96040.0|01_live_animals|
+---------------+----+--

## Library Functions

In [ ]:
from pathlib import Path
from shutil import rmtree

def save_rdd_to_file(rdd, filename: str):
        out_dir = Path("output_dir")
        file = Path("output_dir") / filename
        if file.exists():
            rmtree(out_dir)
        rdd.coalesce(1).saveAsTextFile(str(file))


def save_rdd(fn):

    def wrapper(*argv,**kwargs):
        dname = fn.__name__
        rdd = fn()
        save_rdd_to_file(rdd, dname)
        return rdd

    return wrapper

---

# Using `RDD`

## **1. The number of transactions involving Brazil**;

In [ ]:
# Map
brazil_transactions = rdd.filter(lambda x: x[0] == "Brazil") \
                         .map(lambda x: (x[0], 1))

 # Reduce
brazil_transactions.reduceByKey(lambda a, b: a + b).take(1)

[('Brazil', 184678)]

**Answer:** The number of transactions involving Brazil are 184748

## **2. The number of transactions per year**;

In [ ]:
@save_rdd
def ex2():
    # Map
    transactions_per_year = rdd.map(lambda x: (x[1], 1))

    # Reduce
    return transactions_per_year.reduceByKey(lambda a, b: a + b) \
                                .sortBy(lambda a: a[0])


ex2().take(5)

[('1988', 30970),
 ('1989', 63865),
 ('1990', 72190),
 ('1991', 83843),
 ('1992', 121059)]

## **3. The most commercialized commodity (summing the quantities) in 2016, per flow type**;

In [ ]:
def sumQuantitiesCom(data):
  com = data[3]
  weight = data[6]
  weight = int(weight) if weight else 0
  flow = data[4]

  return (com, flow), weight

@save_rdd
def ex3():
  year2016 = rdd.filter(lambda a: a[1] == "2016") \

  quantitiesComm = year2016.map(sumQuantitiesCom) \
                           .reduceByKey(lambda a,b: a+b)

  flowGroup = quantitiesComm.map(lambda data: (data[0][1], (data[0][0], data[1]))) \
                            .reduceByKey(lambda a,b: a if b[1] < a[1] else b) \
                            .sortBy(lambda a: a[1][1])
  
  return flowGroup

   
ex3().take(5)

[('Re-Import', ('Lumber, coniferous (softwood) thickness < 6 mm', 51951285)),
 ('Re-Export',
  ('Oils petroleum, bituminous, distillates, except crude', 1452933784)),
 ('Export',
  ('Iron ore, concentrate, not iron pyrites,unagglomerate', 1343444789618)),
 ('Import',
  ('Iron ore, concentrate, not iron pyrites,unagglomerate', 1362436716054))]

## **4. The average of commodity values per year**;

In [ ]:
@save_rdd
def ex4():
  perYear = rdd.map(lambda data: (data[1], (float(data[5]), 1)))

  meanYear = perYear \
            .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
            .map(lambda data: (data[0], (data[1][0] / data[1][1]))) \
            .sortBy(lambda data: data[0])

  return meanYear

ex4().take(5)

[('1988', 14906524.119341299),
 ('1989', 12658458.36696156),
 ('1990', 12931120.22652722),
 ('1991', 12950854.07263576),
 ('1992', 11579386.262822261)]

## **5. The average price of commodities per unit type, year, and category in the export flow in Brazil**;

In [ ]:
@save_rdd
def ex5():
  perYear = rdd.filter(lambda a: a[0].upper() == 'BRAZIL') \
               .filter(lambda a: a[4].upper() == 'EXPORT') \
               .filter(lambda a: a[5].isnumeric()) \
               .map(lambda data: ((data[1], data[7], data[9]), (float(data[5]), 1)))

  meanYear = perYear \
            .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
            .map(lambda data: (data[0], (data[1][0] / data[1][1]))) \
            .sortBy(lambda data: (data[0][0], data[0][2]))

  return meanYear

ex5().take(5)

[(('1989', 'Weight in kilograms', '01_live_animals'), 351970.2),
 (('1989', 'Weight in kilograms', '02_meat_and_edible_meat_offal'),
  14544145.46875),
 (('1989',
   'Weight in kilograms',
   '03_fish_crustaceans_molluscs_aquatic_invertebrates_ne'),
  3150434.7),
 (('1989',
   'Volume in litres',
   '04_dairy_products_eggs_honey_edible_animal_product_nes'),
  210515.16666666666),
 (('1989',
   'Number of items',
   '04_dairy_products_eggs_honey_edible_animal_product_nes'),
  760340.3333333334)]

## **6. The commodity with the highest price per unit type and year**;

In [ ]:
@save_rdd
def ex6_the_commodity_with_the_highest_price_per_unit_type_and_year():
    """
    Map:
        map: Mapeando as colunas quantity_name e year, inserindo as instâncias 
        no formato ((quantity_name, year), trade_usd)

        filter: Filtrando pelo elemento trade_usd da tupla

    Reduce:
        reduceByKey: Fazendo reduce e obtendo o maior preço
        sortBy: Ordenando os elementos
    """

    # Map                       
    hp_comm_unit_type_year = rdd.map(lambda x: ((x[7], x[1]), float(x[5]))) \
                                .filter(lambda x: x[1])

    # Reduce
    return hp_comm_unit_type_year.reduceByKey(lambda a, b: a if a > b else b) \
                                 .sortBy(lambda a: a[0])
                                    

# Chamando a função e exibindo 29 instâncias iniciais
ex6_the_commodity_with_the_highest_price_per_unit_type_and_year().take(5)

[(('Area in square metres', '1988'), 403386005.0),
 (('Area in square metres', '1989'), 367370724.0),
 (('Area in square metres', '1990'), 347716597.0),
 (('Area in square metres', '1991'), 403965643.0),
 (('Area in square metres', '1992'), 323730149.0)]

[(('Area in square metres', '1988'), 403386005.0),
 (('Area in square metres', '1989'), 367370724.0),
 (('Area in square metres', '1990'), 347716597.0),
 (('Area in square metres', '1991'), 403965643.0),
 (('Area in square metres', '1992'), 323730149.0)]

## **7. The number of transactions per flow type and year**;

In [ ]:
@save_rdd
def ex7_the_number_of_transactions_per_flow_type_and_year():
    """
    Map:
        map: Mapeando as colunas flow e year, inserindo no formato ((flow, year), 1)
        filter: Filtrando apenas o valor 1 da tupla

    Reduce:
        reduceByKey: Fazendo reduce e obtendo o número de transações
        sortBy: Ordenando os elementos
    """

    # Map                       
    no_trans_flowtype_year = rdd.map(lambda x: ((x[4], x[1]), 1)) \
                                .filter(lambda x: x[1])

    # Reduce
    return no_trans_flowtype_year.reduceByKey(lambda a, b: a + b) \
                                 .sortBy(lambda a: a[0])


# Chamando a função e exibindo 29 instâncias iniciais
ex7_the_number_of_transactions_per_flow_type_and_year().take(5)

[(('Export', '1988'), 12499),
 (('Export', '1989'), 26140),
 (('Export', '1990'), 29139),
 (('Export', '1991'), 32808),
 (('Export', '1992'), 45755)]

---

# Using `SparkSQL`

## **1. The number of transactions involving Brazil**;

In [ ]:
# Método 1: Query SQL

query_ex1 = """ SELECT COUNT(country_or_area)
                FROM table 
                WHERE country_or_area = "Brazil" """

# Executando a Query
df_trans_brazil_1 = spark.sql(query_ex1)

# Exibindo o resultado
df_trans_brazil_1.show(1)

+----------------------+
|count(country_or_area)|
+----------------------+
|                184748|
+----------------------+



In [ ]:
# Método 2: Maneira Clássica de usar o SparkSQL

select_brazil = df.select(f.col('country_or_area')) 
select_brazil[select_brazil['country_or_area'] == 'Brazil'].count()

184748

## **2. The number of transactions per year**;

In [ ]:
# Método 1: Query SQL

query_ex2 = """ SELECT COUNT(year), year
                FROM table
                GROUP BY year
                ORDER BY year ASC """

# Executando a Query
df_trans_per_year = spark.sql(query_ex2)

# Exibindo 29 instâncias iniciais
df_trans_per_year.show(5)

+-----------+----+
|count(year)|year|
+-----------+----+
|      30994|1988|
|      63921|1989|
|      72258|1990|
|      83933|1991|
|     121184|1992|
+-----------+----+
only showing top 5 rows



## **3. The most commercialized commodity (summing the quantities) in 2016, per flow type**;

In [ ]:
# Método 1: Query SQL

query_ex3 = """ SELECT commodity, flow, sum_comm from(
                    SELECT flow, sum_comm, ROW_NUMBER() OVER (PARTITION BY flow ORDER BY sum_comm DESC) rn, commodity 
                        FROM (SELECT SUM(weight_kg) AS sum_comm, flow, commodity
                                    FROM table 
                                    WHERE year = 2016 and comm_code <> 'TOTAL' AND weight_kg IS NOT NULL
                                    GROUP BY flow, commodity)
                        GROUP BY flow, sum_comm, commodity) WHERE rn = 1 
                        ORDER BY sum_comm """

# Executando a Query
df_mc_comm_2016_flowtype = spark.sql(query_ex3)

# Exibindo 29 instâncias iniciais
df_mc_comm_2016_flowtype.show(5)

+--------------------+---------+-------------+
|           commodity|     flow|     sum_comm|
+--------------------+---------+-------------+
|Lumber, coniferou...|Re-Import|     51951285|
|Oils petroleum, b...|Re-Export|   1452933784|
|Iron ore, concent...|   Export|1343444789618|
|Iron ore, concent...|   Import|1362436716054|
+--------------------+---------+-------------+



## **4. The average of commodity values per year**;

In [ ]:
# Método 1: Query SQL

query_ex4 = """ SELECT AVG(trade_usd) AS avgCommValues, year
                FROM table
                WHERE comm_code <> 'TOTAL'
                GROUP BY year
                ORDER BY year ASC """

# Executando a Query
df_avg_comm_values_year = spark.sql(query_ex4)

# Exibindo 29 instâncias iniciais
df_avg_comm_values_year.show(5)

+--------------------+----+
|       avgCommValues|year|
+--------------------+----+
|1.4906524119341299E7|1988|
| 1.265845836696156E7|1989|
| 1.293112022652722E7|1990|
| 1.295085407263576E7|1991|
|1.1579386262822261E7|1992|
+--------------------+----+
only showing top 5 rows



## **5. The average price of commodities per unit type, year, and category in the export flow in Brazil**;

In [ ]:
# Método 1: Query SQL

query_ex5 = """ SELECT AVG(trade_usd) AS avgCommValues, quantity_name, year, category
                FROM table
                WHERE 
                      flow = 'Export' 
                  AND comm_code <> 'TOTAL' 
                  AND country_or_area = 'Brazil' 
                  AND category is not null 
                  AND trade_usd is not null
                GROUP BY quantity_name, year, category
                ORDER BY year, category ASC """

# Executando a Query
df_avg_comm_values_unittype_year_cat_expflow_brazil = spark.sql(query_ex5)

# Exibindo 29 instâncias iniciais
df_avg_comm_values_unittype_year_cat_expflow_brazil.show(5)

+-----------------+-------------------+----+--------------------+
|    avgCommValues|      quantity_name|year|            category|
+-----------------+-------------------+----+--------------------+
|         351970.2|Weight in kilograms|1989|     01_live_animals|
| 1.454414546875E7|Weight in kilograms|1989|02_meat_and_edibl...|
|        3150434.7|Weight in kilograms|1989|03_fish_crustacea...|
|760340.3333333334|    Number of items|1989|04_dairy_products...|
|59596.57142857143|Weight in kilograms|1989|04_dairy_products...|
+-----------------+-------------------+----+--------------------+
only showing top 5 rows



## **6. The commodity with the highest price per unit type and year**;

In [ ]:
# Método 1: Query SQL

query_ex6 = """ SELECT MAX(trade_usd), quantity_name, year
                FROM table
                GROUP BY quantity_name, year 
                ORDER BY quantity_name, year ASC """

# Executando a Query
df_comm_hp_unittype_year_1 = spark.sql(query_ex6)

# Exibindo 29 instâncias iniciais
df_comm_hp_unittype_year_1.show(5)

+--------------+--------------------+----+
|max(trade_usd)|       quantity_name|year|
+--------------+--------------------+----+
|     403386005|Area in square me...|1988|
|     367370724|Area in square me...|1989|
|     347716597|Area in square me...|1990|
|     403965643|Area in square me...|1991|
|     323730149|Area in square me...|1992|
+--------------+--------------------+----+
only showing top 5 rows



## **7. The number of transactions per flow type and year**;

In [ ]:
# Método 1: Query SQL

query_ex7 = """ SELECT COUNT(flow) AS noTransactions, flow, year
                FROM table
                GROUP BY flow, year 
                ORDER BY flow, year ASC """

# Executando a Query
no_trans_flowtype_year_1 = spark.sql(query_ex7)

# Exibindo 29 instâncias iniciais
no_trans_flowtype_year_1.show(5)

+--------------+------+----+
|noTransactions|  flow|year|
+--------------+------+----+
|         12510|Export|1988|
|         26166|Export|1989|
|         29170|Export|1990|
|         32847|Export|1991|
|         45810|Export|1992|
+--------------+------+----+
only showing top 5 rows

